## NFT API Comparison

Trying out Alchemy, Moralis, Simplehash, NFTPort, and anything else I come across over time. As a starting point, I'll test with a super-simple use case - getting the full list of wallets that hold an NFT from a specific collection. This can be the basis of later analysis on those wallets, etc.

## Setup

First, I'll load up the various API keys and URLs for each of the service providers I'm using, and then choose an contract address.

In [ ]:
# General setup
import pandas as pd
import requests
import os

In [1]:
nft_contract_address = '0x13bfb6bd5b9e1e943dc08c1ff7b8fca2d06b20b2'

In [ ]:
# API Keys
sh_api = os.environ["SH_API"]
al_api = os.environ["AL_API"]
ms_api = os.environ["MS_API"]
np_api = os.environ["NP_API"]

# URLs
sh_url = 'https://api.simplehash.com/api/v0/nfts/owners/ethereum/' + nft_contract_address
al_url = 'https://eth-mainnet.g.alchemy.com/nft/v2/' + al_api + '/getOwnersForCollection/?contractAddress=' + nft_contract_address + '&withTokenBalances=true'
ms_url = 'https://deep-index.moralis.io/api/v2/nft/' + nft_contract_address + '/owners?chain=eth&format=decimal'
np_url = 'https://api.nftport.xyz/v0/nfts/' + nft_contract_address + '?chain=ethereum'

# Headers for the ones that need it
sh_headers = {
    "Accept": "application/json",
    "X-API-KEY": sh_api
}

ms_headers = {
    "accept": "application/json",
    "x-api-key": ms_api
}

np_headers = {
    "Authorization": np_api,
    "Content-Type": "application/json"
}

## SimpleHash

In [ ]:
# SimpleHash dataframe

sh_df = pd.DataFrame()

while sh_url != None:
    response = requests.get(sh_url, headers = sh_headers)
    json = response.json()

    if sh_df.empty:
        sh_df = pd.DataFrame(json['owners'])
    else:
        sh_df2 = pd.DataFrame(json['owners'])
        sh_df = pd.concat([sh_df, sh_df2], sort = False)
    sh_url = json['next']

In [ ]:
sh_df

## Alchemy

In [ ]:
response = requests.get(al_url)
json = response.json()
al_df = pd.DataFrame(json['ownerAddresses'])

In [ ]:
al_df

## Moralis

In [ ]:
# Moralis dataframe

import time

ms_df = pd.DataFrame()
ms_url = 'https://deep-index.moralis.io/api/v2/nft/' + nft_contract_address + '/owners?chain=eth&format=decimal'

response = requests.get(ms_url, headers = ms_headers)
json = response.json()
print(json.keys())

ms_df = pd.DataFrame(json['result'])

prev_page = json['page'] - 1
page_size = json['page_size']

print(json['cursor'])

while (prev_page*100+json['page']*100) < json['total']:
    ms_url2 = ms_url + '&cursor=' + json['cursor']
    response = requests.get(ms_url2, headers = ms_headers)
    print(ms_url2)
    json = response.json()
    ms_df2 = pd.DataFrame(json['result'])
    ms_df = pd.concat([ms_df, ms_df2], sort = False)
    prev_page = json['page'] - 1
    page_size = json['page_size']
    time.sleep(1)

In [ ]:
ms_df

ms_df.nunique()

## NFT Port

In [ ]:
np_page = 1
np_url = np_url + '&page_number=' + str(np_page)

# First page of results
response = requests.get(np_url, headers = np_headers)
json = response.json()
np_total = json['total']
np_remaining = np_total

np_df = pd.DataFrame()

while np_remaining > 0 :
    response = requests.get(np_url, headers = np_headers)
    json = response.json()
    print(json.keys())

    if np_df.empty:
        np_df = pd.DataFrame(json['nfts'])
    else:
        np_df2 = pd.DataFrame(json['nfts'])
        np_df = pd.concat([np_df, np_df2], sort = False)
    np_remaining = np_remaining - len(json['nfts'])
    time.sleep(1)

In [ ]:
np_df

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=9b890fc1-911b-465c-9362-c122b6eafef2' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>